### trying different similarity scores

After assigning parent class prediction sets to the input we will further try to assign probabilities or scores to the standard names belonging to the parent classes in the prediction sets.

Some scores are 

1. cosine similarities between the input embeddings and standard name embeddings.
2. tf_idf similarities score with fuzzy match

In [1]:
import pandas as pd
import re
from fuzzywuzzy import fuzz
import numpy as np
import collections

from transformers import AutoTokenizer

/Users/amruthakaruturi/anaconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_excel('standard_names copy.xlsx')
df

,Analysis,Attribute,Standard names,Analysis1,Attribute1
0,CE-SDS (Non-Reduced),HHL,CE-SDS (Non-Reduced) HHL,Capillary Electrophoresis-Sodium Dodecyl Sulf...,Heavy-Heavy-Light (HHL) Chains
1,CE-SDS (Non-Reduced),Purity,CE-SDS (Non-Reduced) Purity,Capillary Electrophoresis-Sodium Dodecyl Sulfa...,Purity
2,CE-SDS (Reduced),Purity,CE-SDS (Reduced) Purity,Capillary Electrophoresis-Sodium Dodecyl Sulfa...,Purity
3,SDS-PAGE (Non-Reduced),Purity,SDS-PAGE (Non-Reduced) Purity,Sodium Dodecyl Sulfate-Polyacrylamide Gel Elec...,Purity
4,SDS-PAGE (Reduced),Purity,SDS-PAGE (Reduced) Purity,Sodium Dodecyl Sulfate-Polyacrylamide Gel Elec...,Purity
5,RP-HPLC,Purity (Main Peak),RP-HPLC Purity,Reverse-Phase High-Performance Liquid Chromato...,Purity (Main Peak)
6,IEF,Acidic Peaks,IEF Acidic Peaks,Isoelectric Focusing (IEF),Acidic Peaks
7,IEF,Basic Peaks,IEF Basic Peaks,Isoelectric Focusing (IEF),Basic Peaks
8,IEF,Main Peak,IEF Main Peak,Isoelectric Focusing (IEF),Main Peak
9,iCIEF,Acidic Peaks,iCIEF Acidic Peaks,Isoelectric Focusing Capillary Electrophoresis...,Acidic Peaks


In [45]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
df1 = df[['Analysis1','Attribute1','Standard names']]
df1['Analysis_tokens']=''
df1['Attribute_tokens']=''
for idx, attr in df1.iterrows():
    df1.at[idx, "Analysis_tokens"] = " ".join(tokenizer.tokenize(attr["Analysis1"]))
    df1.at[idx, "Attribute_tokens"] = " ".join(tokenizer.tokenize(attr["Attribute1"]))
df1['tokens'] = df1.apply(lambda row: row["Analysis_tokens"] +' '+ row['Attribute_tokens'], axis=1)
df1

/Users/amruthakaruturi/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/var/folders/mh/p31c9s4j7x96gcg6jjhwgmd00000gn/T/ipykernel_41254/2084103337.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Analysis_tokens']=''


,Analysis1,Attribute1,Standard names,Analysis_tokens,Attribute_tokens,tokens
0,Capillary Electrophoresis-Sodium Dodecyl Sulf...,Heavy-Heavy-Light (HHL) Chains,CE-SDS (Non-Reduced) HHL,cap ##illa ##ry electro ##ph ##ores ##is - sod...,heavy - heavy - light ( h ##hl ) chains,cap ##illa ##ry electro ##ph ##ores ##is - sod...
1,Capillary Electrophoresis-Sodium Dodecyl Sulfa...,Purity,CE-SDS (Non-Reduced) Purity,cap ##illa ##ry electro ##ph ##ores ##is - sod...,purity,cap ##illa ##ry electro ##ph ##ores ##is - sod...
2,Capillary Electrophoresis-Sodium Dodecyl Sulfa...,Purity,CE-SDS (Reduced) Purity,cap ##illa ##ry electro ##ph ##ores ##is - sod...,purity,cap ##illa ##ry electro ##ph ##ores ##is - sod...
3,Sodium Dodecyl Sulfate-Polyacrylamide Gel Elec...,Purity,SDS-PAGE (Non-Reduced) Purity,sodium dod ##ec ##yl sulfate - poly ##ac ##ryl...,purity,sodium dod ##ec ##yl sulfate - poly ##ac ##ryl...
4,Sodium Dodecyl Sulfate-Polyacrylamide Gel Elec...,Purity,SDS-PAGE (Reduced) Purity,sodium dod ##ec ##yl sulfate - poly ##ac ##ryl...,purity,sodium dod ##ec ##yl sulfate - poly ##ac ##ryl...
5,Reverse-Phase High-Performance Liquid Chromato...,Purity (Main Peak),RP-HPLC Purity,reverse - phase high - performance liquid ch #...,purity ( main peak ),reverse - phase high - performance liquid ch #...
6,Isoelectric Focusing (IEF),Acidic Peaks,IEF Acidic Peaks,iso ##ele ##ctric focusing ( ie ##f ),acidic peaks,iso ##ele ##ctric focusing ( ie ##f ) acidic p...
7,Isoelectric Focusing (IEF),Basic Peaks,IEF Basic Peaks,iso ##ele ##ctric focusing ( ie ##f ),basic peaks,iso ##ele ##ctric focusing ( ie ##f ) basic peaks
8,Isoelectric Focusing (IEF),Main Peak,IEF Main Peak,iso ##ele ##ctric focusing ( ie ##f ),main peak,iso ##ele ##ctric focusing ( ie ##f ) main peak
9,Isoelectric Focusing Capillary Electrophoresis...,Acidic Peaks,iCIEF Acidic Peaks,iso ##ele ##ctric focusing cap ##illa ##ry ele...,acidic peaks,iso ##ele ##ctric focusing cap ##illa ##ry ele...


In [13]:
label = [0,0,0,0,0,1,2,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,4,4,5,6,7,8,9,10]
len(label)

32

In [46]:
df1['label']=label

In [4]:
df2 = df1[['Standard names','tokens']]
df2 

,Standard names,tokens
0,CE-SDS (Non-Reduced) HHL,cap ##illa ##ry electro ##ph ##ores ##is - sod...
1,CE-SDS (Non-Reduced) Purity,cap ##illa ##ry electro ##ph ##ores ##is - sod...
2,CE-SDS (Reduced) Purity,cap ##illa ##ry electro ##ph ##ores ##is - sod...
3,SDS-PAGE (Non-Reduced) Purity,sodium dod ##ec ##yl sulfate - poly ##ac ##ryl...
4,SDS-PAGE (Reduced) Purity,sodium dod ##ec ##yl sulfate - poly ##ac ##ryl...
5,RP-HPLC Purity,reverse - phase high - performance liquid ch #...
6,IEF Acidic Peaks,iso ##ele ##ctric focusing ( ie ##f ) acidic p...
7,IEF Basic Peaks,iso ##ele ##ctric focusing ( ie ##f ) basic peaks
8,IEF Main Peak,iso ##ele ##ctric focusing ( ie ##f ) main peak
9,iCIEF Acidic Peaks,iso ##ele ##ctric focusing cap ##illa ##ry ele...


In [6]:
df2['label']=label
df2

/var/folders/mh/p31c9s4j7x96gcg6jjhwgmd00000gn/T/ipykernel_41254/3286806690.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['label']=label


,Standard names,tokens,label
0,CE-SDS (Non-Reduced) HHL,cap ##illa ##ry electro ##ph ##ores ##is - sod...,0
1,CE-SDS (Non-Reduced) Purity,cap ##illa ##ry electro ##ph ##ores ##is - sod...,0
2,CE-SDS (Reduced) Purity,cap ##illa ##ry electro ##ph ##ores ##is - sod...,0
3,SDS-PAGE (Non-Reduced) Purity,sodium dod ##ec ##yl sulfate - poly ##ac ##ryl...,0
4,SDS-PAGE (Reduced) Purity,sodium dod ##ec ##yl sulfate - poly ##ac ##ryl...,0
5,RP-HPLC Purity,reverse - phase high - performance liquid ch #...,1
6,IEF Acidic Peaks,iso ##ele ##ctric focusing ( ie ##f ) acidic p...,2
7,IEF Basic Peaks,iso ##ele ##ctric focusing ( ie ##f ) basic peaks,2
8,IEF Main Peak,iso ##ele ##ctric focusing ( ie ##f ) main peak,2
9,iCIEF Acidic Peaks,iso ##ele ##ctric focusing cap ##illa ##ry ele...,2


In [8]:
l1= pd.read_csv('predicted_labels.csv')
l1.head()

,analysis,Attribute_name,class_probabilities,labels,max_prob,label
0,D_250475,IL2 INHIBITION ASSAY,[0.9641308188438416],[5],0.964131,5
1,D_95007196,PH,[0.997627317905426],[8],0.997627,8
2,D_M00003744,ABATACEPT MAJOR BAND (REDUCED),[0.9975391626358032],[0],0.997539,0
3,Y_SM_95011468_R,BIOASSAY,"[0.8629494905471802, 0.04849901795387268, 0.01...","[5, 9, 6, 10, 2]",0.862949,5
4,250684_CE_SDS_REDUC,SUM HEAVY AND LIGHT CHAIN,[0.9992501139640808],[0],0.999250,0


In [15]:
ana = 'BIOLOGICAL_ASSAY'
att = 'Assay As Is'
predicted_classes= [5, 9, 3, 10, 1, 6, 8]
parent_class_probs = [0.7642839550971985, 0.07047382742166519, 0.047781892120838165, 0.02137192152440548, 0.018122196197509766, 0.014519959688186646, 0.013605629093945026]
candidates = df1[df1['label'].isin(predicted_classes)]

In [16]:
candidates

,Analysis1,Attribute1,Standard names,Analysis_tokens,Attribute_tokens,tokens,label
5,Reverse-Phase High-Performance Liquid Chromato...,Purity (Main Peak),RP-HPLC Purity,reverse - phase high - performance liquid ch #...,purity ( main peak ),reverse - phase high - performance liquid ch #...,1
18,Size Exclusion High-Performance Liquid Chromat...,High Molecular Weight(HMW),SE-HPLC HMW,size exclusion high - performance liquid ch ##...,high molecular weight ( hm ##w ),size exclusion high - performance liquid ch ##...,3
19,Size Exclusion High-Performance Liquid Chromat...,Low Molecular Weight(LMW),SE-HPLC LMW,size exclusion high - performance liquid ch ##...,low molecular weight ( l ##m ##w ),size exclusion high - performance liquid ch ##...,3
20,Size Exclusion High-Performance Liquid Chromat...,Monomer,SE-HPLC Monomer,size exclusion high - performance liquid ch ##...,mono ##mer,size exclusion high - performance liquid ch ##...,3
21,Size Exclusion Ultra-Performance Liquid Chroma...,High Molecular Weight(HMW),SE-UPLC HMW,size exclusion ultra - performance liquid ch #...,high molecular weight ( hm ##w ),size exclusion ultra - performance liquid ch #...,3
22,Size Exclusion Ultra-Performance Liquid Chroma...,Low Molecular Weight(LMW),SE-UPLC LMW,size exclusion ultra - performance liquid ch #...,low molecular weight ( l ##m ##w ),size exclusion ultra - performance liquid ch #...,3
23,Size Exclusion Ultra-Performance Liquid Chroma...,Monomer,SE-UPLC Monomer,size exclusion ultra - performance liquid ch #...,mono ##mer,size exclusion ultra - performance liquid ch #...,3
26,Cell-Based Bioassay,Potency,Potency by Cell-Based Bioassay,cell - based bio ##ass ##ay,pot ##ency,cell - based bio ##ass ##ay pot ##ency,5
27,Enzyme-Linked Immunosorbent Assay (ELISA),Binding activity or Potency,Potency by Binding ELISA,enzyme - linked im ##mun ##oso ##rb ##ent ass ...,binding activity or pot ##ency,enzyme - linked im ##mun ##oso ##rb ##ent ass ...,6
29,potential of Hydrogen (pH),potential of Hydrogen (pH),pH,potential of hydrogen ( ph ),potential of hydrogen ( ph ),potential of hydrogen ( ph ) potential of hydr...,8


In [17]:
candidate_terms = candidates['Analysis1'] + " " + candidates['Attribute1']
test_text = f"{att} {ana}"

In [18]:
candidate_terms

5     Reverse-Phase High-Performance Liquid Chromato...
18    Size Exclusion High-Performance Liquid Chromat...
19    Size Exclusion High-Performance Liquid Chromat...
20    Size Exclusion High-Performance Liquid Chromat...
21    Size Exclusion Ultra-Performance Liquid Chroma...
22    Size Exclusion Ultra-Performance Liquid Chroma...
23    Size Exclusion Ultra-Performance Liquid Chroma...
26                          Cell-Based Bioassay Potency
27    Enzyme-Linked Immunosorbent Assay (ELISA) Bind...
29    potential of Hydrogen (pH) potential of Hydrog...
30     Absorbance at 280 nanometers (nm) (A280) Prot...
31                        Polysorbate 80 Polysorbate 80
dtype: object

In [19]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import pandas as pd
import numpy as np

# Load pre-trained DistilBERT model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def embed_text(text, tokenizer, model):
    """
    Generate embeddings for a given text using DistilBERT.
    Args:
        text (str): Input text.
        tokenizer: DistilBERT tokenizer.
        model: DistilBERT model.
    Returns:
        torch.Tensor: Embedding vector for the text.
    """
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        output = model(**tokens)
    return output.last_hidden_state.mean(dim=1)

In [20]:
# Step 2: Calculate Textual Similarity using DistilBERT
test_embedding = embed_text(test_text, tokenizer, model)
candidate_embeddings = torch.cat(
    [embed_text(term, tokenizer, model) for term in candidate_terms.tolist()]
)
similarities = torch.nn.functional.cosine_similarity(
    test_embedding, candidate_embeddings
)
# Normalize similarity scores
normalized_similarities = similarities / torch.max(similarities)

tensor([0.6788, 0.6961, 0.6883, 0.6830, 0.6927, 0.6851, 0.6850, 0.7576, 0.7564,
        0.6237, 0.6555, 0.6704])

In [22]:
test_embedding.shape

torch.Size([1, 768])

In [23]:
# Normalize similarity scores
normalized_similarities = similarities / torch.max(similarities)

In [24]:
normalized_similarities

tensor([0.8960, 0.9188, 0.9084, 0.9015, 0.9143, 0.9043, 0.9041, 1.0000, 0.9983,
        0.8232, 0.8652, 0.8848])

In [28]:
candidates

,Analysis1,Attribute1,Standard names,Analysis_tokens,Attribute_tokens,tokens,label
5,Reverse-Phase High-Performance Liquid Chromato...,Purity (Main Peak),RP-HPLC Purity,reverse - phase high - performance liquid ch #...,purity ( main peak ),reverse - phase high - performance liquid ch #...,1
18,Size Exclusion High-Performance Liquid Chromat...,High Molecular Weight(HMW),SE-HPLC HMW,size exclusion high - performance liquid ch ##...,high molecular weight ( hm ##w ),size exclusion high - performance liquid ch ##...,3
19,Size Exclusion High-Performance Liquid Chromat...,Low Molecular Weight(LMW),SE-HPLC LMW,size exclusion high - performance liquid ch ##...,low molecular weight ( l ##m ##w ),size exclusion high - performance liquid ch ##...,3
20,Size Exclusion High-Performance Liquid Chromat...,Monomer,SE-HPLC Monomer,size exclusion high - performance liquid ch ##...,mono ##mer,size exclusion high - performance liquid ch ##...,3
21,Size Exclusion Ultra-Performance Liquid Chroma...,High Molecular Weight(HMW),SE-UPLC HMW,size exclusion ultra - performance liquid ch #...,high molecular weight ( hm ##w ),size exclusion ultra - performance liquid ch #...,3
22,Size Exclusion Ultra-Performance Liquid Chroma...,Low Molecular Weight(LMW),SE-UPLC LMW,size exclusion ultra - performance liquid ch #...,low molecular weight ( l ##m ##w ),size exclusion ultra - performance liquid ch #...,3
23,Size Exclusion Ultra-Performance Liquid Chroma...,Monomer,SE-UPLC Monomer,size exclusion ultra - performance liquid ch #...,mono ##mer,size exclusion ultra - performance liquid ch #...,3
26,Cell-Based Bioassay,Potency,Potency by Cell-Based Bioassay,cell - based bio ##ass ##ay,pot ##ency,cell - based bio ##ass ##ay pot ##ency,5
27,Enzyme-Linked Immunosorbent Assay (ELISA),Binding activity or Potency,Potency by Binding ELISA,enzyme - linked im ##mun ##oso ##rb ##ent ass ...,binding activity or pot ##ency,enzyme - linked im ##mun ##oso ##rb ##ent ass ...,6
29,potential of Hydrogen (pH),potential of Hydrogen (pH),pH,potential of hydrogen ( ph ),potential of hydrogen ( ph ),potential of hydrogen ( ph ) potential of hydr...,8


In [47]:
prob_mapping

{5: 0.7642839550971985,
 9: 0.07047382742166519,
 3: 0.047781892120838165,
 10: 0.02137192152440548,
 1: 0.018122196197509766,
 6: 0.014519959688186646,
 8: 0.013605629093945026}

In [31]:
prob_mapping = dict(zip(predicted_classes, parent_class_probs))
candidates['parent_class_prob'] = candidates['label'].map(prob_mapping)
candidates

/var/folders/mh/p31c9s4j7x96gcg6jjhwgmd00000gn/T/ipykernel_41254/2100591185.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidates['parent_class_prob'] = candidates['label'].map(prob_mapping)


,Analysis1,Attribute1,Standard names,Analysis_tokens,Attribute_tokens,tokens,label,parent_class_prob
5,Reverse-Phase High-Performance Liquid Chromato...,Purity (Main Peak),RP-HPLC Purity,reverse - phase high - performance liquid ch #...,purity ( main peak ),reverse - phase high - performance liquid ch #...,1,0.018122
18,Size Exclusion High-Performance Liquid Chromat...,High Molecular Weight(HMW),SE-HPLC HMW,size exclusion high - performance liquid ch ##...,high molecular weight ( hm ##w ),size exclusion high - performance liquid ch ##...,3,0.047782
19,Size Exclusion High-Performance Liquid Chromat...,Low Molecular Weight(LMW),SE-HPLC LMW,size exclusion high - performance liquid ch ##...,low molecular weight ( l ##m ##w ),size exclusion high - performance liquid ch ##...,3,0.047782
20,Size Exclusion High-Performance Liquid Chromat...,Monomer,SE-HPLC Monomer,size exclusion high - performance liquid ch ##...,mono ##mer,size exclusion high - performance liquid ch ##...,3,0.047782
21,Size Exclusion Ultra-Performance Liquid Chroma...,High Molecular Weight(HMW),SE-UPLC HMW,size exclusion ultra - performance liquid ch #...,high molecular weight ( hm ##w ),size exclusion ultra - performance liquid ch #...,3,0.047782
22,Size Exclusion Ultra-Performance Liquid Chroma...,Low Molecular Weight(LMW),SE-UPLC LMW,size exclusion ultra - performance liquid ch #...,low molecular weight ( l ##m ##w ),size exclusion ultra - performance liquid ch #...,3,0.047782
23,Size Exclusion Ultra-Performance Liquid Chroma...,Monomer,SE-UPLC Monomer,size exclusion ultra - performance liquid ch #...,mono ##mer,size exclusion ultra - performance liquid ch #...,3,0.047782
26,Cell-Based Bioassay,Potency,Potency by Cell-Based Bioassay,cell - based bio ##ass ##ay,pot ##ency,cell - based bio ##ass ##ay pot ##ency,5,0.764284
27,Enzyme-Linked Immunosorbent Assay (ELISA),Binding activity or Potency,Potency by Binding ELISA,enzyme - linked im ##mun ##oso ##rb ##ent ass ...,binding activity or pot ##ency,enzyme - linked im ##mun ##oso ##rb ##ent ass ...,6,0.014520
29,potential of Hydrogen (pH),potential of Hydrogen (pH),pH,potential of hydrogen ( ph ),potential of hydrogen ( ph ),potential of hydrogen ( ph ) potential of hydr...,8,0.013606


In [40]:
candidates['cosine_similarity_score'] = normalized_similarities.numpy()
candidates

/var/folders/mh/p31c9s4j7x96gcg6jjhwgmd00000gn/T/ipykernel_41254/3614880555.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidates['cosine_similarity_score'] = normalized_similarities.numpy()


,Analysis1,Attribute1,Standard names,Analysis_tokens,Attribute_tokens,tokens,label,parent_class_prob,similarity_score,combined_score,cosine_similarity_score
5,Reverse-Phase High-Performance Liquid Chromato...,Purity (Main Peak),RP-HPLC Purity,reverse - phase high - performance liquid ch #...,purity ( main peak ),reverse - phase high - performance liquid ch #...,1,0.018122,0.896002,0.369274,0.896002
18,Size Exclusion High-Performance Liquid Chromat...,High Molecular Weight(HMW),SE-HPLC HMW,size exclusion high - performance liquid ch ##...,high molecular weight ( hm ##w ),size exclusion high - performance liquid ch ##...,3,0.047782,0.918754,0.396171,0.918754
19,Size Exclusion High-Performance Liquid Chromat...,Low Molecular Weight(LMW),SE-HPLC LMW,size exclusion high - performance liquid ch ##...,low molecular weight ( l ##m ##w ),size exclusion high - performance liquid ch ##...,3,0.047782,0.908417,0.392036,0.908417
20,Size Exclusion High-Performance Liquid Chromat...,Monomer,SE-HPLC Monomer,size exclusion high - performance liquid ch ##...,mono ##mer,size exclusion high - performance liquid ch ##...,3,0.047782,0.901468,0.389256,0.901468
21,Size Exclusion Ultra-Performance Liquid Chroma...,High Molecular Weight(HMW),SE-UPLC HMW,size exclusion ultra - performance liquid ch #...,high molecular weight ( hm ##w ),size exclusion ultra - performance liquid ch #...,3,0.047782,0.914297,0.394388,0.914297
22,Size Exclusion Ultra-Performance Liquid Chroma...,Low Molecular Weight(LMW),SE-UPLC LMW,size exclusion ultra - performance liquid ch #...,low molecular weight ( l ##m ##w ),size exclusion ultra - performance liquid ch #...,3,0.047782,0.904311,0.390394,0.904311
23,Size Exclusion Ultra-Performance Liquid Chroma...,Monomer,SE-UPLC Monomer,size exclusion ultra - performance liquid ch #...,mono ##mer,size exclusion ultra - performance liquid ch #...,3,0.047782,0.904121,0.390317,0.904121
26,Cell-Based Bioassay,Potency,Potency by Cell-Based Bioassay,cell - based bio ##ass ##ay,pot ##ency,cell - based bio ##ass ##ay pot ##ency,5,0.764284,1.000000,0.858570,1.000000
27,Enzyme-Linked Immunosorbent Assay (ELISA),Binding activity or Potency,Potency by Binding ELISA,enzyme - linked im ##mun ##oso ##rb ##ent ass ...,binding activity or pot ##ency,enzyme - linked im ##mun ##oso ##rb ##ent ass ...,6,0.014520,0.998329,0.408044,0.998329
29,potential of Hydrogen (pH),potential of Hydrogen (pH),pH,potential of hydrogen ( ph ),potential of hydrogen ( ph ),potential of hydrogen ( ph ) potential of hydr...,8,0.013606,0.823221,0.337452,0.823221


In [39]:
candidates['combined_score'] = ( 0.6 * candidates['parent_class_prob'] + 0.4 * candidates['similarity_score'])
candidates

/var/folders/mh/p31c9s4j7x96gcg6jjhwgmd00000gn/T/ipykernel_41254/2554766018.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidates['combined_score'] = ( 0.6 * candidates['parent_class_prob'] + 0.4 * candidates['similarity_score'])


,Analysis1,Attribute1,Standard names,Analysis_tokens,Attribute_tokens,tokens,label,parent_class_prob,similarity_score,combined_score
5,Reverse-Phase High-Performance Liquid Chromato...,Purity (Main Peak),RP-HPLC Purity,reverse - phase high - performance liquid ch #...,purity ( main peak ),reverse - phase high - performance liquid ch #...,1,0.018122,0.896002,0.369274
18,Size Exclusion High-Performance Liquid Chromat...,High Molecular Weight(HMW),SE-HPLC HMW,size exclusion high - performance liquid ch ##...,high molecular weight ( hm ##w ),size exclusion high - performance liquid ch ##...,3,0.047782,0.918754,0.396171
19,Size Exclusion High-Performance Liquid Chromat...,Low Molecular Weight(LMW),SE-HPLC LMW,size exclusion high - performance liquid ch ##...,low molecular weight ( l ##m ##w ),size exclusion high - performance liquid ch ##...,3,0.047782,0.908417,0.392036
20,Size Exclusion High-Performance Liquid Chromat...,Monomer,SE-HPLC Monomer,size exclusion high - performance liquid ch ##...,mono ##mer,size exclusion high - performance liquid ch ##...,3,0.047782,0.901468,0.389256
21,Size Exclusion Ultra-Performance Liquid Chroma...,High Molecular Weight(HMW),SE-UPLC HMW,size exclusion ultra - performance liquid ch #...,high molecular weight ( hm ##w ),size exclusion ultra - performance liquid ch #...,3,0.047782,0.914297,0.394388
22,Size Exclusion Ultra-Performance Liquid Chroma...,Low Molecular Weight(LMW),SE-UPLC LMW,size exclusion ultra - performance liquid ch #...,low molecular weight ( l ##m ##w ),size exclusion ultra - performance liquid ch #...,3,0.047782,0.904311,0.390394
23,Size Exclusion Ultra-Performance Liquid Chroma...,Monomer,SE-UPLC Monomer,size exclusion ultra - performance liquid ch #...,mono ##mer,size exclusion ultra - performance liquid ch #...,3,0.047782,0.904121,0.390317
26,Cell-Based Bioassay,Potency,Potency by Cell-Based Bioassay,cell - based bio ##ass ##ay,pot ##ency,cell - based bio ##ass ##ay pot ##ency,5,0.764284,1.000000,0.858570
27,Enzyme-Linked Immunosorbent Assay (ELISA),Binding activity or Potency,Potency by Binding ELISA,enzyme - linked im ##mun ##oso ##rb ##ent ass ...,binding activity or pot ##ency,enzyme - linked im ##mun ##oso ##rb ##ent ass ...,6,0.014520,0.998329,0.408044
29,potential of Hydrogen (pH),potential of Hydrogen (pH),pH,potential of hydrogen ( ph ),potential of hydrogen ( ph ),potential of hydrogen ( ph ) potential of hydr...,8,0.013606,0.823221,0.337452


In [38]:

ranked_candidates = candidates.sort_values(by='combined_score', ascending=False)
ranked_candidates



,Analysis1,Attribute1,Standard names,Analysis_tokens,Attribute_tokens,tokens,label,parent_class_prob,similarity_score,combined_score
26,Cell-Based Bioassay,Potency,Potency by Cell-Based Bioassay,cell - based bio ##ass ##ay,pot ##ency,cell - based bio ##ass ##ay pot ##ency,5,0.764284,1.000000,0.834999
27,Enzyme-Linked Immunosorbent Assay (ELISA),Binding activity or Potency,Potency by Binding ELISA,enzyme - linked im ##mun ##oso ##rb ##ent ass ...,binding activity or pot ##ency,enzyme - linked im ##mun ##oso ##rb ##ent ass ...,6,0.014520,0.998329,0.309663
18,Size Exclusion High-Performance Liquid Chromat...,High Molecular Weight(HMW),SE-HPLC HMW,size exclusion high - performance liquid ch ##...,high molecular weight ( hm ##w ),size exclusion high - performance liquid ch ##...,3,0.047782,0.918754,0.309074
30,Absorbance at 280 nanometers (nm) (A280),Protein Concentration,Protein Concentration (A280),absorb ##ance at 280 nano ##meter ##s ( nm ) (...,protein concentration,absorb ##ance at 280 nano ##meter ##s ( nm ) (...,9,0.070474,0.865209,0.308894
21,Size Exclusion Ultra-Performance Liquid Chroma...,High Molecular Weight(HMW),SE-UPLC HMW,size exclusion ultra - performance liquid ch #...,high molecular weight ( hm ##w ),size exclusion ultra - performance liquid ch #...,3,0.047782,0.914297,0.307736
19,Size Exclusion High-Performance Liquid Chromat...,Low Molecular Weight(LMW),SE-HPLC LMW,size exclusion high - performance liquid ch ##...,low molecular weight ( l ##m ##w ),size exclusion high - performance liquid ch ##...,3,0.047782,0.908417,0.305972
22,Size Exclusion Ultra-Performance Liquid Chroma...,Low Molecular Weight(LMW),SE-UPLC LMW,size exclusion ultra - performance liquid ch #...,low molecular weight ( l ##m ##w ),size exclusion ultra - performance liquid ch #...,3,0.047782,0.904311,0.304741
23,Size Exclusion Ultra-Performance Liquid Chroma...,Monomer,SE-UPLC Monomer,size exclusion ultra - performance liquid ch #...,mono ##mer,size exclusion ultra - performance liquid ch #...,3,0.047782,0.904121,0.304684
20,Size Exclusion High-Performance Liquid Chromat...,Monomer,SE-HPLC Monomer,size exclusion high - performance liquid ch ##...,mono ##mer,size exclusion high - performance liquid ch ##...,3,0.047782,0.901468,0.303888
5,Reverse-Phase High-Performance Liquid Chromato...,Purity (Main Peak),RP-HPLC Purity,reverse - phase high - performance liquid ch #...,purity ( main peak ),reverse - phase high - performance liquid ch #...,1,0.018122,0.896002,0.281486


In [41]:
def compute_doc_term_matrix(vocabulary, collection, threshold=80):
    """
    Computes a document-term matrix with fuzzy matching.

    Parameters:
    - vocabulary (list): List of vocabulary words.
    - collection (list of list of str): List of documents, where each document is a list of words.
    - threshold (int): Minimum fuzz.ratio score to consider a match.

    Returns:
    - np.ndarray: Document-term matrix.
    """
    # Initialize the document-term matrix
    doc_term_matrix = np.zeros((len(collection), len(vocabulary))) 
    vocab_indices = {word: idx for idx, word in enumerate(vocabulary)}

    for i, line in enumerate(collection):
        # Count the frequencies of words in the document
        dictfreq = collections.Counter(line)
        
        for word, freq in dictfreq.items():
            # Find the best match for the word in the vocabulary using fuzz.ratio
            best_match = None
            best_score = 0
            for vocab_word in vocabulary:
                score = fuzz.ratio(word, vocab_word)
                if score > best_score and score >= threshold:
                    best_match = vocab_word
                    best_score = score
            
            # If a match is found, update the matrix
            if best_match:
                index = vocab_indices[best_match]
                doc_term_matrix[i, index] += freq

    return doc_term_matrix


def compute_tf_matrix(dtm):
    # fill in your code here
    tfm = np.zeros(dtm.shape)   
    nwords = dtm.sum(axis=1,keepdims=True) # total no.of words in each document
    epsilon = 1e-9
    tfm = dtm/(nwords + epsilon) #to avoid zeroes in denominator
    return tfm
def compute_idf(doc_term_matrix):
    idf = np.zeros(doc_term_matrix.shape[1]) 
    idf = np.log10(doc_term_matrix.shape[0] / np.where(np.sum(doc_term_matrix > 0, axis=0) > 0, np.sum(doc_term_matrix > 0, axis=0), 1e-9))  
    return idf

def compute_tf_idf_matrix(document_term_matrix,idf):
    tf_idf = document_term_matrix * idf
    return tf_idf
def softmax(count):
    exp_count = np.exp(count - np.max(count))  # Subtracting max for numerical stability
    return exp_count / exp_count.sum()

In [42]:
def m2(label,ana,att,threshold=80):
    thre = threshold
    ana_tokens = " ".join(tokenizer.tokenize(ana))
    att_tokens = " ".join(tokenizer.tokenize(att))
    tokens = ana_tokens + ' ' + att_tokens

    t1 = clean_text(tokens)
    lexicon ={}
    for i in range(len(t1)):
        lexicon[t1[i]]=i
    
    l2 = df2[df2['label'] == label]['tokens1'].tolist()
    doc_term_matrix = compute_doc_term_matrix(lexicon,l2)
    idf = compute_idf(doc_term_matrix)
    tf_idf_matrix = compute_tf_idf_matrix(doc_term_matrix,idf)
    #n = df2[df2['label'] == label].shape[0]
    #l = df2[df2['label'] == label]['tokens1'].tolist()
    labels = df2[df2['label']==label]['Standard names']
    a = tf_idf_matrix.sum(axis=1)
    prob = softmax(a)
    return prob,a,labels

In [43]:
candidates

,Analysis1,Attribute1,Standard names,Analysis_tokens,Attribute_tokens,tokens,label,parent_class_prob,similarity_score,combined_score,cosine_similarity_score
5,Reverse-Phase High-Performance Liquid Chromato...,Purity (Main Peak),RP-HPLC Purity,reverse - phase high - performance liquid ch #...,purity ( main peak ),reverse - phase high - performance liquid ch #...,1,0.018122,0.896002,0.369274,0.896002
18,Size Exclusion High-Performance Liquid Chromat...,High Molecular Weight(HMW),SE-HPLC HMW,size exclusion high - performance liquid ch ##...,high molecular weight ( hm ##w ),size exclusion high - performance liquid ch ##...,3,0.047782,0.918754,0.396171,0.918754
19,Size Exclusion High-Performance Liquid Chromat...,Low Molecular Weight(LMW),SE-HPLC LMW,size exclusion high - performance liquid ch ##...,low molecular weight ( l ##m ##w ),size exclusion high - performance liquid ch ##...,3,0.047782,0.908417,0.392036,0.908417
20,Size Exclusion High-Performance Liquid Chromat...,Monomer,SE-HPLC Monomer,size exclusion high - performance liquid ch ##...,mono ##mer,size exclusion high - performance liquid ch ##...,3,0.047782,0.901468,0.389256,0.901468
21,Size Exclusion Ultra-Performance Liquid Chroma...,High Molecular Weight(HMW),SE-UPLC HMW,size exclusion ultra - performance liquid ch #...,high molecular weight ( hm ##w ),size exclusion ultra - performance liquid ch #...,3,0.047782,0.914297,0.394388,0.914297
22,Size Exclusion Ultra-Performance Liquid Chroma...,Low Molecular Weight(LMW),SE-UPLC LMW,size exclusion ultra - performance liquid ch #...,low molecular weight ( l ##m ##w ),size exclusion ultra - performance liquid ch #...,3,0.047782,0.904311,0.390394,0.904311
23,Size Exclusion Ultra-Performance Liquid Chroma...,Monomer,SE-UPLC Monomer,size exclusion ultra - performance liquid ch #...,mono ##mer,size exclusion ultra - performance liquid ch #...,3,0.047782,0.904121,0.390317,0.904121
26,Cell-Based Bioassay,Potency,Potency by Cell-Based Bioassay,cell - based bio ##ass ##ay,pot ##ency,cell - based bio ##ass ##ay pot ##ency,5,0.764284,1.000000,0.858570,1.000000
27,Enzyme-Linked Immunosorbent Assay (ELISA),Binding activity or Potency,Potency by Binding ELISA,enzyme - linked im ##mun ##oso ##rb ##ent ass ...,binding activity or pot ##ency,enzyme - linked im ##mun ##oso ##rb ##ent ass ...,6,0.014520,0.998329,0.408044,0.998329
29,potential of Hydrogen (pH),potential of Hydrogen (pH),pH,potential of hydrogen ( ph ),potential of hydrogen ( ph ),potential of hydrogen ( ph ) potential of hydr...,8,0.013606,0.823221,0.337452,0.823221


In [ ]:
ana = 'SDS'
att = 'SDS-PAGE (nR) Purity'
prob,a,labels = m2(0,ana,att)
print(f'prob:{prob}')
print(f'score:{a}')
print(labels)

In [56]:
df1

,Analysis1,Attribute1,Standard names,Analysis_tokens,Attribute_tokens,tokens,label
0,Capillary Electrophoresis-Sodium Dodecyl Sulf...,Heavy-Heavy-Light (HHL) Chains,CE-SDS (Non-Reduced) HHL,cap ##illa ##ry electro ##ph ##ores ##is - sod...,heavy - heavy - light ( h ##hl ) chains,cap ##illa ##ry electro ##ph ##ores ##is - sod...,0
1,Capillary Electrophoresis-Sodium Dodecyl Sulfa...,Purity,CE-SDS (Non-Reduced) Purity,cap ##illa ##ry electro ##ph ##ores ##is - sod...,purity,cap ##illa ##ry electro ##ph ##ores ##is - sod...,0
2,Capillary Electrophoresis-Sodium Dodecyl Sulfa...,Purity,CE-SDS (Reduced) Purity,cap ##illa ##ry electro ##ph ##ores ##is - sod...,purity,cap ##illa ##ry electro ##ph ##ores ##is - sod...,0
3,Sodium Dodecyl Sulfate-Polyacrylamide Gel Elec...,Purity,SDS-PAGE (Non-Reduced) Purity,sodium dod ##ec ##yl sulfate - poly ##ac ##ryl...,purity,sodium dod ##ec ##yl sulfate - poly ##ac ##ryl...,0
4,Sodium Dodecyl Sulfate-Polyacrylamide Gel Elec...,Purity,SDS-PAGE (Reduced) Purity,sodium dod ##ec ##yl sulfate - poly ##ac ##ryl...,purity,sodium dod ##ec ##yl sulfate - poly ##ac ##ryl...,0
5,Reverse-Phase High-Performance Liquid Chromato...,Purity (Main Peak),RP-HPLC Purity,reverse - phase high - performance liquid ch #...,purity ( main peak ),reverse - phase high - performance liquid ch #...,1
6,Isoelectric Focusing (IEF),Acidic Peaks,IEF Acidic Peaks,iso ##ele ##ctric focusing ( ie ##f ),acidic peaks,iso ##ele ##ctric focusing ( ie ##f ) acidic p...,2
7,Isoelectric Focusing (IEF),Basic Peaks,IEF Basic Peaks,iso ##ele ##ctric focusing ( ie ##f ),basic peaks,iso ##ele ##ctric focusing ( ie ##f ) basic peaks,2
8,Isoelectric Focusing (IEF),Main Peak,IEF Main Peak,iso ##ele ##ctric focusing ( ie ##f ),main peak,iso ##ele ##ctric focusing ( ie ##f ) main peak,2
9,Isoelectric Focusing Capillary Electrophoresis...,Acidic Peaks,iCIEF Acidic Peaks,iso ##ele ##ctric focusing cap ##illa ##ry ele...,acidic peaks,iso ##ele ##ctric focusing cap ##illa ##ry ele...,2


In [70]:
import pandas as pd
import numpy as np
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from sklearn.feature_extraction.text import TfidfVectorizer

# Load DistilBERT model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def embed_text(text, tokenizer, model):
    """
    Generate embeddings for a given text using DistilBERT.
    Args:
        text (str): Input text.
        tokenizer: DistilBERT tokenizer.
        model: DistilBERT model.
    Returns:
        torch.Tensor: Embedding vector for the text.
    """
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        output = model(**tokens)
    return output.last_hidden_state.mean(dim=1)

def compute_tfidf(df1, candidates):
    """
    Computes the TF-IDF scores for candidates in df1.
    Args:
        df1 (pd.DataFrame): Standard dataset with 'attribute' and 'analysis'.
        candidates (pd.DataFrame): DataFrame of selected candidates for a test row.
    Returns:
        np.ndarray: TF-IDF scores for the candidates.
    """
    # Combine 'attribute' and 'analysis' for TF-IDF calculation
    candidate_terms = candidates['Attribute1'] + " " + candidates['Analysis1']
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(candidate_terms)
    return tfidf_matrix

def classify_with_prediction_set(df1, test_df, threshold=0.5):
    """
    Classify test data into a prediction set of standard names from df1 based on a combined score.
    Rows where no candidate meets the threshold will be labeled as "No Match".
    
    Args:
        df1 (pd.DataFrame): Standard dataset with columns ['analysis', 'attribute', 'standard_name', 'label'].
        test_df (pd.DataFrame): Test dataset with columns ['analysis', 'attribute', 'predict_set', 'classprobabilities', 'max_prob'].
        threshold (float): Minimum score required to include a candidate in the prediction set.
    
    Returns:
        pd.Series: Prediction sets for test_df.
    """
    prediction_sets = []
    for i, test_row in test_df.iterrows():
        # Step 1: Subset Selection
        predicted_classes = test_row['predict_set']  # Predicted parent classes (list)
        parent_class_probs = test_row['classprobabilities']  # Probabilities as a dictionary
        candidates = df1[df1['label'].isin(predicted_classes)]
        if candidates.empty:
            prediction_sets.append("No Match")
            continue

        # Combine attribute and analysis for candidate matching
        candidate_terms = candidates['Attribute1'] + " " + candidates['Analysis1']
        test_text = f"{test_row['attribute']} {test_row['analysis']}"
        
        # Step 2: Calculate Textual Similarity using DistilBERT
        test_embedding = embed_text(test_text, tokenizer, model)
        candidate_embeddings = torch.cat(
            [embed_text(term, tokenizer, model) for term in candidate_terms.tolist()]
        )
        similarities = torch.nn.functional.cosine_similarity(
            test_embedding, candidate_embeddings
        )
        
        # Normalize similarity scores
        normalized_similarities = similarities / torch.max(similarities)
        
        # Step 3: Compute TF-IDF Scores
        tfidf_matrix = compute_tfidf(df1, candidates)
        tfidf_scores = tfidf_matrix.sum(axis=1).A.flatten()  # Summing the TF-IDF scores for each candidate
        
        # Step 4: Combine Scores
        candidates['parent_class_prob'] = candidates['label'].map(test_row['classprobabilities'])
        candidates['similarity_score'] = normalized_similarities.numpy()
        candidates['tfidf_score'] = tfidf_scores
        candidates['combined_score'] = (
            0.5 * candidates['parent_class_prob'] + 0.3 * candidates['similarity_score'] + 0.2 * candidates['tfidf_score']
        )
        
        # Step 5: Select Candidates Above Threshold
        valid_candidates = candidates[candidates['combined_score'] >= threshold]
        if valid_candidates.empty:
            prediction_sets.append("No Match")
        else:
            prediction_sets.append(valid_candidates['Standard names'].tolist())
    
    return pd.Series(prediction_sets), valid_candidates



In [67]:
candidates

,Analysis1,Attribute1,Standard names,Analysis_tokens,Attribute_tokens,tokens,label,parent_class_prob,similarity_score,combined_score,cosine_similarity_score
5,Reverse-Phase High-Performance Liquid Chromato...,Purity (Main Peak),RP-HPLC Purity,reverse - phase high - performance liquid ch #...,purity ( main peak ),reverse - phase high - performance liquid ch #...,1,0.018122,0.896002,0.369274,0.896002
18,Size Exclusion High-Performance Liquid Chromat...,High Molecular Weight(HMW),SE-HPLC HMW,size exclusion high - performance liquid ch ##...,high molecular weight ( hm ##w ),size exclusion high - performance liquid ch ##...,3,0.047782,0.918754,0.396171,0.918754
19,Size Exclusion High-Performance Liquid Chromat...,Low Molecular Weight(LMW),SE-HPLC LMW,size exclusion high - performance liquid ch ##...,low molecular weight ( l ##m ##w ),size exclusion high - performance liquid ch ##...,3,0.047782,0.908417,0.392036,0.908417
20,Size Exclusion High-Performance Liquid Chromat...,Monomer,SE-HPLC Monomer,size exclusion high - performance liquid ch ##...,mono ##mer,size exclusion high - performance liquid ch ##...,3,0.047782,0.901468,0.389256,0.901468
21,Size Exclusion Ultra-Performance Liquid Chroma...,High Molecular Weight(HMW),SE-UPLC HMW,size exclusion ultra - performance liquid ch #...,high molecular weight ( hm ##w ),size exclusion ultra - performance liquid ch #...,3,0.047782,0.914297,0.394388,0.914297
22,Size Exclusion Ultra-Performance Liquid Chroma...,Low Molecular Weight(LMW),SE-UPLC LMW,size exclusion ultra - performance liquid ch #...,low molecular weight ( l ##m ##w ),size exclusion ultra - performance liquid ch #...,3,0.047782,0.904311,0.390394,0.904311
23,Size Exclusion Ultra-Performance Liquid Chroma...,Monomer,SE-UPLC Monomer,size exclusion ultra - performance liquid ch #...,mono ##mer,size exclusion ultra - performance liquid ch #...,3,0.047782,0.904121,0.390317,0.904121
26,Cell-Based Bioassay,Potency,Potency by Cell-Based Bioassay,cell - based bio ##ass ##ay,pot ##ency,cell - based bio ##ass ##ay pot ##ency,5,0.764284,1.000000,0.858570,1.000000
27,Enzyme-Linked Immunosorbent Assay (ELISA),Binding activity or Potency,Potency by Binding ELISA,enzyme - linked im ##mun ##oso ##rb ##ent ass ...,binding activity or pot ##ency,enzyme - linked im ##mun ##oso ##rb ##ent ass ...,6,0.014520,0.998329,0.408044,0.998329
29,potential of Hydrogen (pH),potential of Hydrogen (pH),pH,potential of hydrogen ( ph ),potential of hydrogen ( ph ),potential of hydrogen ( ph ) potential of hydr...,8,0.013606,0.823221,0.337452,0.823221


In [53]:
ana = 'BIOLOGICAL_ASSAY'
att = 'Assay As Is'
predicted_classes= [5, 9, 3, 10, 1, 6, 8]
parent_class_probs = [0.7642839550971985, 0.07047382742166519, 0.047781892120838165, 0.02137192152440548, 0.018122196197509766, 0.014519959688186646, 0.013605629093945026]
max_prob = 0.764283955097198
# Test dataset
test_df = pd.DataFrame({
    'analysis': [ana],
    'attribute': [att],
    'predict_set': [predicted_classes],
    'classprobabilities': [dict(zip(predicted_classes, parent_class_probs))],
    'max_prob': [max_prob]
})


In [54]:
test_df

,analysis,attribute,predict_set,classprobabilities,max_prob
0,BIOLOGICAL_ASSAY,Assay As Is,"[5, 9, 3, 10, 1, 6, 8]","{5: 0.7642839550971985, 9: 0.07047382742166519...",0.764284


In [71]:

# Run classification
predicted_set , vcandidates = classify_with_prediction_set(df1, test_df, threshold=0.5)
vcandidates


/var/folders/mh/p31c9s4j7x96gcg6jjhwgmd00000gn/T/ipykernel_41254/3652042461.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidates['parent_class_prob'] = candidates['label'].map(test_row['classprobabilities'])
/var/folders/mh/p31c9s4j7x96gcg6jjhwgmd00000gn/T/ipykernel_41254/3652042461.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidates['similarity_score'] = normalized_similarities.numpy()
/var/folders/mh/p31c9s4j7x96gcg6jjhwgmd00000gn/T/ipykernel_41254/3652042461.py:87: SettingWithCopyW

,Analysis1,Attribute1,Standard names,Analysis_tokens,Attribute_tokens,tokens,label,parent_class_prob,similarity_score,tfidf_score,combined_score
5,Reverse-Phase High-Performance Liquid Chromato...,Purity (Main Peak),RP-HPLC Purity,reverse - phase high - performance liquid ch #...,purity ( main peak ),reverse - phase high - performance liquid ch #...,1,0.018122,0.874869,3.182176,0.907957
18,Size Exclusion High-Performance Liquid Chromat...,High Molecular Weight(HMW),SE-HPLC HMW,size exclusion high - performance liquid ch ##...,high molecular weight ( hm ##w ),size exclusion high - performance liquid ch ##...,3,0.047782,0.906893,3.160267,0.928012
19,Size Exclusion High-Performance Liquid Chromat...,Low Molecular Weight(LMW),SE-HPLC LMW,size exclusion high - performance liquid ch ##...,low molecular weight ( l ##m ##w ),size exclusion high - performance liquid ch ##...,3,0.047782,0.897641,3.401141,0.973411
20,Size Exclusion High-Performance Liquid Chromat...,Monomer,SE-HPLC Monomer,size exclusion high - performance liquid ch ##...,mono ##mer,size exclusion high - performance liquid ch ##...,3,0.047782,0.893783,2.953999,0.882826
21,Size Exclusion Ultra-Performance Liquid Chroma...,High Molecular Weight(HMW),SE-UPLC HMW,size exclusion ultra - performance liquid ch #...,high molecular weight ( hm ##w ),size exclusion ultra - performance liquid ch #...,3,0.047782,0.899386,3.413469,0.976401
22,Size Exclusion Ultra-Performance Liquid Chroma...,Low Molecular Weight(LMW),SE-UPLC LMW,size exclusion ultra - performance liquid ch #...,low molecular weight ( l ##m ##w ),size exclusion ultra - performance liquid ch #...,3,0.047782,0.891203,3.397561,0.970764
23,Size Exclusion Ultra-Performance Liquid Chroma...,Monomer,SE-UPLC Monomer,size exclusion ultra - performance liquid ch #...,mono ##mer,size exclusion ultra - performance liquid ch #...,3,0.047782,0.894237,2.944004,0.880963
26,Cell-Based Bioassay,Potency,Potency by Cell-Based Bioassay,cell - based bio ##ass ##ay,pot ##ency,cell - based bio ##ass ##ay pot ##ency,5,0.764284,1.000000,1.995996,1.081341
27,Enzyme-Linked Immunosorbent Assay (ELISA),Binding activity or Potency,Potency by Binding ELISA,enzyme - linked im ##mun ##oso ##rb ##ent ass ...,binding activity or pot ##ency,enzyme - linked im ##mun ##oso ##rb ##ent ass ...,6,0.014520,0.978933,2.996957,0.900331
29,potential of Hydrogen (pH),potential of Hydrogen (pH),pH,potential of hydrogen ( ph ),potential of hydrogen ( ph ),potential of hydrogen ( ph ) potential of hydr...,8,0.013606,0.809405,2.000000,0.649624


In [ ]:
ana = 'BIOLOGICAL_ASSAY'
att = 'Assay As Is'
predicted_classes= [5, 9, 3, 10, 1, 6, 8]
parent_class_probs = [0.7642839550971985, 0.07047382742166519, 0.047781892120838165, 0.02137192152440548, 0.018122196197509766, 0.014519959688186646, 0.013605629093945026]


In [72]:
l1.head()

,analysis,Attribute_name,class_probabilities,labels,max_prob,label
0,D_250475,IL2 INHIBITION ASSAY,[0.9641308188438416],[5],0.964131,5
1,D_95007196,PH,[0.997627317905426],[8],0.997627,8
2,D_M00003744,ABATACEPT MAJOR BAND (REDUCED),[0.9975391626358032],[0],0.997539,0
3,Y_SM_95011468_R,BIOASSAY,"[0.8629494905471802, 0.04849901795387268, 0.01...","[5, 9, 6, 10, 2]",0.862949,5
4,250684_CE_SDS_REDUC,SUM HEAVY AND LIGHT CHAIN,[0.9992501139640808],[0],0.999250,0


In [76]:
label

[0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 5,
 6,
 7,
 8,
 9,
 10]

In [85]:
l1[(l1['max_prob']<0.9) & (l1['max_prob']>0.8)].sort_values(by=['max_prob'])[:30]

,analysis,Attribute_name,class_probabilities,labels,max_prob,label
1465,J_TC-I-I53,Methionine oxidation,"[0.8003227114677429, 0.11585094779729843, 0.02...","[0, 8, 7, 5]",0.800323,0
1410,250593,PEAK M3,"[0.8006750345230103, 0.0913228690624237, 0.032...","[2, 1, 10, 4]",0.800675,2
553,249699_ID_ABATAC,MIG TIME DIFFERENCE SSS1RWS BELATACEPT,"[0.8021142482757568, 0.06770172715187073, 0.03...","[2, 1, 3, 4, 7]",0.802114,2
855,A_250343,METHIONINE OXIDATION 1DECPT,"[0.8024576902389526, 0.16617564857006073]","[0, 8]",0.802458,0
474,D_250658,PEAK 3,"[0.8026857376098633, 0.09904810041189194, 0.02...","[2, 1, 10, 4]",0.802686,2
4535,BIOLOGICAL_ASSAY,Potency (FACS) Purity Relative Potency,"[0.8032898902893066, 0.09551434963941574, 0.05...","[0, 5, 6]",0.803290,0
654,249699_ID_ABATAC,MIG TIME DIFFERENCE SSS1RWS ABATACEPT,"[0.8039323687553406, 0.06693276762962341, 0.03...","[2, 1, 7, 4, 3]",0.803932,2
361,249699,RESULT BOTTLE COMPARED WITH SSS2,"[0.8065735101699829, 0.07424280792474747, 0.02...","[1, 7, 2, 0, 4, 9, 5]",0.806574,1
133,Y_M00000176_NR_CESDS,NonRed: HHL,"[0.8081291913986206, 0.12432004511356354, 0.01...","[0, 8, 7, 1]",0.808129,0
568,Y_MTD10212_BIA,Bioassay,"[0.8096074461936951, 0.08842746168375015, 0.02...","[5, 9, 7, 6, 10, 2]",0.809607,5


In [74]:
l1[l1['max_prob']>0.9]

,analysis,Attribute_name,class_probabilities,labels,max_prob,label
0,D_250475,IL2 INHIBITION ASSAY,[0.9641308188438416],[5],0.964131,5
1,D_95007196,PH,[0.997627317905426],[8],0.997627,8
2,D_M00003744,ABATACEPT MAJOR BAND (REDUCED),[0.9975391626358032],[0],0.997539,0
4,250684_CE_SDS_REDUC,SUM HEAVY AND LIGHT CHAIN,[0.9992501139640808],[0],0.999250,0
5,250580_TOTAL_PROT,PAAD,[0.9724606275558472],[9],0.972461,9
...,...,...,...,...,...,...
7725,CE_SDS,CE-SDS (Red.) Rel. Retention Time: Undefined P...,[0.9993515610694885],[0],0.999352,0
7726,CE_SDS,CE-SDS (Red.) Rel. Retention Time: Undefined P...,[0.999354898929596],[0],0.999355,0
7727,CE_SDS,CE-SDS (Red.) Rel. Retention Time: Undefined P...,[0.9993543028831482],[0],0.999354,0
7728,CE_SDS_2,CE-SDS (Non-reduced) RRT of Unknown Minor Peaks,[0.9994910955429077],[0],0.999491,0


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class StandardNameClassifier(nn.Module):
    def _init_(self, embedding_dim, num_standard_names, num_parent_classes):
        super(StandardNameClassifier, self)._init_()

        self.embedding_dim = embedding_dim
        self.num_standard_names = num_standard_names
        self.num_parent_classes = num_parent_classes
        
        # Define the layers
        self.input_projection = nn.Linear(embedding_dim, 256)  # Transform input embedding
        self.standard_embedding_layer = nn.Embedding(num_standard_names, embedding_dim)  # Embeddings for standard names
        
        # Final classification layer
        self.fc = nn.Linear(256, num_standard_names)

    def forward(self, input_embedding, parent_class_onehot, parent_class_probabilities):
        # Step 1: Process input embedding
        input_embedding_transformed = F.relu(self.input_projection(input_embedding))  # (batch_size, 256)

        # Step 2: Standard name embeddings
        standard_name_embeddings = self.standard_embedding_layer.weight  # (num_standard_names, embedding_dim)

        # Step 3: Compute the attention weights for standard names based on parent class
        # Compute weighted embeddings for the prediction set
        weighted_standard_names = torch.matmul(parent_class_onehot, standard_name_embeddings)  # (batch_size, num_standard_names, embedding_dim)
        
        # Weighted by parent class probabilities
        weighted_embeddings = weighted_standard_names * parent_class_probabilities.unsqueeze(-1)  # (batch_size, num_standard_names, embedding_dim)
        
        # Step 4: Combine input embedding and weighted standard name embeddings
        combined_embeddings = input_embedding_transformed.unsqueeze(1) + weighted_embeddings  # (batch_size, num_standard_names, 256 + embedding_dim)
        
        # Step 5: Flatten and pass through a final classifier
        combined_embeddings_flat = combined_embeddings.view(combined_embeddings.size(0), -1)  # (batch_size, num_standard_names * embedding_dim)
        
        output_logits = self.fc(combined_embeddings_flat)  # (batch_size, num_standard_names)
        
        # Step 6: Apply Softmax to get probabilities
        output_probs = F.softmax(output_logits, dim=-1)  # (batch_size, num_standard_names)
        
        return output_probs